In [4]:
from tqdm.notebook import tqdm as progress
import pandas as pd
import plotly.express as px
from plotly.offline import plot
import dimcli
from dimcli.utils import *
import os, sys, time, json
from IPython.display import Image
from IPython.core.display import HTML 
import csv
import ast


/Users/FionaXu/Documents/Research/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning:

urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020



In [6]:
#Log into Dimensions
#API key is: 9F8D648F0D7E437CB1736BEBDF007F02
#!pip install dimcli -U --quiet 

print("==\nLogging in..")
# https://digital-science.github.io/dimcli/getting-started.html#authentication
ENDPOINT = "https://app.dimensions.ai"
if 'google.colab' in sys.modules:
    import getpass
    dimcli.login(key=KEY, endpoint=ENDPOINT)
else:
    KEY = "9F8D648F0D7E437CB1736BEBDF007F02"
    dimcli.login(key=KEY, endpoint=ENDPOINT)
dsl = dimcli.Dsl()

==
Logging in..
Dimcli - Dimensions API Client (v1.4)
Connected to: <https://app.dimensions.ai/api/dsl> - DSL v2.10
Method: manual login


In [8]:
#RUN ONLY IF YOUR NAMES ARE IN A CSV, IF NOT MOVE TO NEXT CELL
def read_names_from_csv(file_path, encoding='utf-8'):
    all_names = []

    with open(file_path, 'r', encoding=encoding, errors='ignore') as csv_file:
        csv_reader = csv.reader(csv_file)
        
        # Assuming the first column contains first names and the second column contains last names
        for row in csv_reader:
            try:
                if len(row) >= 2:  # Ensure the row has at least two columns
                    first_names = row[1]
                    last_names = row[3]
                    full_name = first_names + " " + last_names
                    all_names.append(full_name)
            except  UnicodeDecodeError as e:
                print(f"Error decoding line {csv_reader.line_num}: {e}")

    return all_names

# Example usage
file_path = r'150physicianscientists.csv'
all_names = read_names_from_csv(file_path)
all_names = all_names[1:]

# Now you can use the lists first_names and last_names as needed
print(f'ALL NAMES({len(all_names)}) :')
print("======================================================")
print(all_names)


ALL NAMES(150) :
['Sarki Abdulkadir', 'Ash Alizadeh', 'Vineet Arora', 'Andrew Auerbach', 'Adam Bass', 'Edward Behrens', 'Rameen Beroukhim', 'Karl Bilimoria', 'Ron Bose', 'Russell Bowler', 'Nicole Calakos', 'Sarat Chandarlapaty', 'Ping Chi', 'Ryan Corcoran', 'Peter Crompton', 'Aaron Cypess', 'Maximilian Diehn', 'Justin Dimick', 'Mark Dransfield', 'Brian Feldman', 'Ariel Feldstein', 'Martin Fernandez-Zapico', 'Alessia Fornoni', 'Terry Fry', 'Rene Galindo', 'Anupam Agarwal', 'Jayakrishna Ambati', 'Suneel Apte', 'Kyong-Mi Chang', 'Herbert Cohen', 'Laurence Cooper', 'George Cotsarelis', 'Kenneth Covinsky', 'Maurizio Del Poeta', 'Phillip Dennis', 'Michael Diamond', 'Jay Edelberg', 'David Engman', 'Daniel Fowler', 'Vance Fowler', 'John Frangioni', 'Bin Gao', 'Michael Glickman', 'Stephen Hammes', 'Hal Hoffman', 'Sam Hwang', 'S. Karumanchi', 'Douglas Kerr', 'Tejvir Khurana', 'Fadlo Khuri', 'Maria Abreu', 'Michael Ackerman', 'David Andes', 'Scott Armstrong', 'R. Barr', 'Marcel Behr', 'Daniel Ben

In [ ]:
#RUN CELL IF YOUR NAMES ARE IN A LIST
#put your list of names here
all_names = []

In [9]:
#gets research id for a name
def researcherIds(name):
    res = dsl.query_iterative(f"""search researchers for "{name}" return researchers""")
    listDict = res.json['researchers']
    allIds = set()
    for idv in listDict:
        allIds.add(idv['id'])

    allIds = list(allIds)  # Convert set back to a list if needed
    
    if len(allIds) > 512:    
        allIds = allIds[0:512]
    print(f'Total of {len(allIds)} unique ids for {name} ')

    # Constructing the DSL query with the list of IDs
    query_ids = ', '.join([f'"{id_val}"' for id_val in allIds])
    query = f'search researchers where id in [{query_ids}] return researchers[id+obsolete+redirect]'

    # Execute the DSL query
    res2 = dsl.query_iterative(query)
    listDict2 = res2.json['researchers']
    allWorkingIDs = set()
    for person in listDict2:
        if person['obsolete'] == 0:  # Current Working Id(s)
            allWorkingIDs.add(person["id"])
        else:
            for ids in person['redirect']:
                allWorkingIDs.add(ids)
    
    print(f'++++++++ FINAL {len(allWorkingIDs)} Working ID(s) +++++++++++')  
    print(allWorkingIDs)
    
    return allWorkingIDs
testIDs = researcherIds("Leslie Schoenfield")

Starting iteration with limit=1000 skip=0 ...
0-2 / 2 (0.33s)
===
Records extracted: 2
Starting iteration with limit=1000 skip=0 ...


Total of 2 unique ids for Leslie Schoenfield 


0-2 / 2 (4.50s)
===
Records extracted: 2


++++++++ FINAL 2 Working ID(s) +++++++++++
{'ur.014424615212.32', 'ur.0102031123.62'}


In [10]:
def allIds(namesList):
    finalNames = {} 
    secondNames = {}
    zeroNames = {}
    count = 0

    for name in namesList:
        try:
            validIds = researcherIds(name)

            if len(validIds) == 0:
                zeroNames[name] = set()
                finalNames[name] = set()
                pass
            elif len(validIds) == 1:
                count += 1
                print(name, count)
                finalNames[name] = validIds
            elif len(validIds) == 2:
                secondNames[name] = validIds
                finalNames[name] = validIds
            else:
                finalNames[name] = validIds
        except Exception as e:
            print(f"An error occurred for {name}: {e}")
    return finalNames, secondNames, zeroNames
        
resultAll, pairsAll, zeroNames = allIds(all_names)
#resultAll is all names with more than two ids
#pairsAll is all names with two ids
#zeroNames is all names with no ids

Starting iteration with limit=1000 skip=0 ...
0-1 / 1 (0.24s)
===
Records extracted: 1
Starting iteration with limit=1000 skip=0 ...


Total of 1 unique ids for Sarki Abdulkadir 


0-1 / 1 (0.25s)
===
Records extracted: 1
Starting iteration with limit=1000 skip=0 ...


++++++++ FINAL 1 Working ID(s) +++++++++++
{'ur.01151331063.92'}
Sarki Abdulkadir 1


0-2 / 2 (2.84s)
===
Records extracted: 2
Starting iteration with limit=1000 skip=0 ...


...

++++++++ FINAL 3 Working ID(s) +++++++++++
{'ur.01267631365.29', 'ur.0612074377.64', 'ur.014503114362.31'}


0-8 / 8 (0.27s)
===
Records extracted: 8
Starting iteration with limit=1000 skip=0 ...


Total of 8 unique ids for David Golan 


0-8 / 8 (4.54s)
===
Records extracted: 8


++++++++ FINAL 8 Working ID(s) +++++++++++
{'ur.015601425747.72', 'ur.013411104347.36', 'ur.016377006347.90', 'ur.0770777565.58', 'ur.012764675327.78', 'ur.012106750212.06', 'ur.01144174070.48', 'ur.07423033552.33'}


In [11]:
#this writes only names with more than two ids to file
print(len(resultAll))
print(len(pairsAll))
print(len(zeroNames))

def write_set_to_csv(file_path, my_set):
    with open(file_path, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)

        # Write the header
        writer.writerow(['Name', 'ID'])

        # Write the data
        for name, id_set in my_set.items():                
            id_set = list(id_set)
            writer.writerow([name, id_set])
            
write_set_to_csv("dimensionsAI_NameIds.csv", resultAll)


149
15
0


In [12]:
#if you want to write all you names to a csv file
merged_dict = {resultAll, pairsAll, zeroNames}
write_set_to_csv("dimensionsAI_NameIds.csv", merged_dict)

TypeError: unhashable type: 'dict'